In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

from scipy.spatial.distance import cdist

import joblib
from sklearn import set_config
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics  import silhouette_score
set_config(display="diagram")

In [3]:
data = pd.read_csv("../data/final_data.csv", index_col=0)
data.head(5)

,brand,img_url,laptop_name,display_size,processor_type,graphics_card,disk_space,discount_price,ratings_5max,disk_type
0,HP,https://encrypted-tbn0.gstatic.com/images?q=tb...,Notebook 14-df0008nx,14.0,Intel Celeron N4000,Intel HD Graphics 600,64.0,1259.0,0.0,eMMC
1,Lenovo,https://encrypted-tbn0.gstatic.com/images?q=tb...,IdeaPad 330S-14IKB,14.0,Intel Core i5-8250U,Intel UHD Graphics 620,1024.0,1849.0,3.3,HDD
2,Huawei,https://encrypted-tbn0.gstatic.com/images?q=tb...,MateBook D Volta,14.0,Intel Core i5-8250U,NVIDIA GeForce MX150 (2 GB),256.0,2999.0,0.0,SSD
3,Dell,https://encrypted-tbn0.gstatic.com/images?q=tb...,Inspiron 15 3567,15.6,Intel Core i3-7020U,Intel HD Graphics 620,1024.0,1849.0,0.0,HDD
4,Asus,https://encrypted-tbn0.gstatic.com/images?q=tb...,VivoBook 15 X510UR,15.6,Intel Core i7-8550U,NVIDIA GeForce 930MX (2 GB),1024.0,2499.0,0.0,HDD


In [11]:
df = data.pivot_table(index='brand', columns="graphics_card", values='ratings_5max').fillna(0)
df.head()

graphics_card,AMD Radeon 520 (2 GB),AMD Radeon 530 (2 GB),AMD Radeon 530 (4 GB),AMD Radeon 530M (4 GB),AMD Radeon 535 (2 GB),AMD Radeon 540 (4 GB),AMD Radeon R5,AMD Radeon R5-M340 (2 GB),AMD Radeon R5-M430 (2 GB),AMD Radeon R7-M445 (4 GB),...,NVIDIA GeForce MX130 (2 GB),NVIDIA GeForce MX130 (4 GB),NVIDIA GeForce MX150 (2 GB),NVIDIA GeForce MX150 (4 GB),NVIDIA GeForce MX230 (2 GB),NVIDIA GeForce MX250 (2 GB),NVIDIA GeForce MX250 (4 GB),Radeon Pro 555 GDDR5 (2 GB),Radeon Pro 555X GDDR5 (4 GB),Radeon Pro 560X GDDR5 (4 GB)
brand,,,,,,,,,,,,,,,,,,,,,
Acer,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.875,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.000000
Apple,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.000000,0.000,0.0,0.0,0.0,4.6,0.0,2.933333
Asus,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.733333,0.000,0.0,0.0,0.0,0.0,0.0,0.000000
Dell,0.0,1.333333,3.8,0.0,0.0,0.0,0.0,0.0,3.3,0.0,...,0.000,3.7,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.000000
HP,0.0,3.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,1.600000,2.225,0.0,0.0,0.0,0.0,0.0,0.000000


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()
print(X.shape)

(4, 9)


In [32]:
vectorizer = TfidVectorizer()
vectorizer.fit_transform()

array([0.        , 0.46979139, 0.58028582, 0.38408524, 0.        ,
       0.        , 0.38408524, 0.        , 0.38408524])